# api 크롤링

기존의 크롤링 방식은 

1. 내가 직접 홈페이지에 브라우저를 켜서 접근하거나(selenium)

혹은 

2. 파이썬 자체적으로 호출을 요청해서(requests)

데이터를 가지고 오는 방식이었습니다.

이 방식은 서버에 부하가 많이 갈 뿐더러 나에게 필요없는 데이터까지 한 번에 호출해서 가져오는 문제가 있습니다.

따라서 서비스 제공자 측에서는 서버 부하를 줄이고, 사용자에게 맞춤형의 데이터를 제공하기 위해 api서버를 운영합니다.

api서버는 인가된 데이터만을 개발자에게 넘겨서 서버도 안정적으로 유지하며(횟수 제한이 있는 사이트도 많습니다. 라이엇 데이터는 1초에 5회, 2분에 100회로 제한됩니다.)

필요없는 동영상 자료나 그림 자료를 호출하지 않으므로 트래픽을 줄일 수 있습니다.


api 서버 접근시 보통 urllib.request 를 이용하게 됩니다.

In [1]:
# 사이트에 자료 요청
import urllib.request
#json 데이터 핸들링
import json
# DataFrame 자료형 활용
import pandas as pd 
# json 데이터를 pandas DataFrame으로 변환
from pandas.io.json import json_normalize

# 영진위 api 신청

https://www.kobis.or.kr/kobisopenapi/homepg/main/main.do

1. 접속 후 가입 및 로그인 

2. 키발급 탭에서 키 발급하기 

3. 요청 api종류 보고 요청양식 및 데이터 확인하기 

In [3]:
api_key = "5c6ac9b9844cd60b13a5ea12c2eede98" # 발급받은 키만 복붙하세요.
request_date = "20220802" # 조회날짜를  xxxxyydd 20220802 형식으로 넣어주세요.
url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
print(url)

https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=5c6ac9b9844cd60b13a5ea12c2eede98&targetDt=20220802


# json데이터를 팬더스 데이터프레임으로 변환

api 데이터는 보통 json(JavaScript Object Notation) 타입으로 받아집니다.

쉽게 말하면 자바스크립트 데이터를 전달하기 좋게 설정한 자료형인데

파이썬의 딕셔너리와 거의 같다고 보시면 됩니다.

따라서 json 데이터를 팬더스 데이터로 변환한다는 것은 사실상

딕셔너리 데이터를 팬더스 데이터프레임으로 변환하는것입니다.

In [4]:
# 위의 url 변수를 이용해 데이터를 요청합니다.
r = urllib.request.urlopen(url)

# 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
json_raw =r.read()

In [5]:
# b'~~~~~~~'로 시작하는 이 데이터는 파이썬에서 byte자료형으로 처리되고
# 데이터프레임으로는 바로 변환이 불가능합니다.
json_raw

b'{"boxOfficeResult":{"boxofficeType":"\xec\x9d\xbc\xeb\xb3\x84 \xeb\xb0\x95\xec\x8a\xa4\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4","showRange":"20220802~20220802","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20209343","movieNm":"\xed\x95\x9c\xec\x82\xb0: \xec\x9a\xa9\xec\x9d\x98 \xec\xb6\x9c\xed\x98\x84","openDt":"2022-07-27","salesAmt":"3418297205","salesShare":"56.5","salesInten":"-445775729","salesChange":"-11.5","salesAcc":"30676698831","audiCnt":"340707","audiInten":"-45277","audiChange":"-11.7","audiAcc":"2996787","scrnCnt":"2030","showCnt":"9659"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20205362","movieNm":"\xeb\xaf\xb8\xeb\x8b\x88\xec\x96\xb8\xec\xa6\x882","openDt":"2022-07-20","salesAmt":"843554512","salesShare":"13.9","salesInten":"-60441616","salesChange":"-6.7","salesAcc":"16439366347","audiCnt":"90863","audiInten":"-6344","audiChange":"-6.5","audiAcc":"1693084","scrnCnt":"945","showCnt":"2837"},{"r

In [6]:
# 바이트자료형임을 확인 
type(json_raw)

bytes

In [7]:
# 따라서 utf-8 형식으로 고쳐줘야 합니다. .decode()를 이용합니다.
# encode => 우리가 쓰는 문자를 컴퓨터가 쓰는 형태로 변환 
# decode => 컴퓨터가 쓰는 형태의 문자를 우리가 쓰는 형태로 변환
json_utf8 = json_raw.decode("utf-8")

In [8]:
# 딕셔너리 구조를 유지하고 있는 문자로 변환됨
type(json_utf8)

str

In [9]:
json_utf8

'{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20220802~20220802","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20209343","movieNm":"한산: 용의 출현","openDt":"2022-07-27","salesAmt":"3418297205","salesShare":"56.5","salesInten":"-445775729","salesChange":"-11.5","salesAcc":"30676698831","audiCnt":"340707","audiInten":"-45277","audiChange":"-11.7","audiAcc":"2996787","scrnCnt":"2030","showCnt":"9659"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20205362","movieNm":"미니언즈2","openDt":"2022-07-20","salesAmt":"843554512","salesShare":"13.9","salesInten":"-60441616","salesChange":"-6.7","salesAcc":"16439366347","audiCnt":"90863","audiInten":"-6344","audiChange":"-6.5","audiAcc":"1693084","scrnCnt":"945","showCnt":"2837"},{"rnum":"3","rank":"3","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20194376","movieNm":"탑건: 매버릭","openDt":"2022-06-22","salesAmt":"678742098","salesShare":"11.2","salesInten":"-52334711"

In [10]:
# json.loads(딕셔너리 형태의 문자열 자료)
# 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
json_complete = json.loads(json_utf8)

In [11]:
# 딕셔너리(json) 변환 완료 확인
type(json_complete)

dict

In [17]:
# json_normalize(딕셔너리 자료)
# 딕셔너리 자료를 데이터프레임 형태로 변환해줌
pd.json_normalize(json_complete) # 보통 json 데이터를 받아오면 list화 되어있는 컬럼이 하나 들어옴. 해당 list 데이터를 타켓해서 데이터 가져오기

,boxOfficeResult.boxofficeType,boxOfficeResult.showRange,boxOfficeResult.dailyBoxOfficeList
0,일별 박스오피스,20220802~20220802,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."


In [18]:
# boxOfficeResult.dailyBoxOfficeList 컬럼 내부에 딕셔너리가 여럿 묶인 리스트가 보이므로
# boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20209343,한산: 용의 출현,2022-07-27,3418297205,56.5,-445775729,-11.5,30676698831,340707,-45277,-11.7,2996787,2030,9659
1,2,2,0,OLD,20205362,미니언즈2,2022-07-20,843554512,13.9,-60441616,-6.7,16439366347,90863,-6344,-6.5,1693084,945,2837
2,3,3,0,OLD,20194376,탑건: 매버릭,2022-06-22,678742098,11.2,-52334711,-7.2,76662106293,66650,-4841,-6.8,7187516,802,2072
3,4,4,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,351444239,5.8,-19020699,-5.1,2356729812,38807,-1971,-4.8,251109,677,1440
4,5,5,0,OLD,20208446,외계+인 1부,2022-07-20,363649895,6.0,-40994799,-10.1,14968915621,36506,-3867,-9.6,1434570,712,1804
5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,190209473,3.1,-155223,-0.1,17660574672,18263,43,0.2,1708584,412,752
6,7,7,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,81394059,1.3,5183642,6.8,4434875457,8088,617,8.3,427204,185,271
7,8,8,1,OLD,20211792,헌트,2022-08-10,29720000,0.5,-6703000,-18.4,93969000,4280,233,5.8,10688,17,17
8,9,9,-1,OLD,20196410,비상선언,2022-08-03,26522000,0.4,-46635300,-63.7,247169216,1681,-3204,-65.6,22329,5,7
9,10,10,0,OLD,20224662,토르: 러브 앤 썬더,2022-07-06,12040299,0.2,-151561,-1.2,29461697403,1082,1,0.1,2711863,38,51


In [19]:
# json_normalize()는 pd.DataFrame()으로 대체 가능합니다.
pd.DataFrame(json_complete).loc['dailyBoxOfficeList']

boxOfficeResult    [{'rnum': '1', 'rank': '1', 'rankInten': '0', ...
Name: dailyBoxOfficeList, dtype: object